In [50]:
import string
import re
from numpy import array, argmax, random, take, delete
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, RepeatVector, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers

import pickle
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 200)

In [5]:
model = load_model('./models/model.h1.en_fr')

In [54]:
# loading
with open('./vocabulary/fr_tokenizer.pickle', 'rb') as handle:
    fr_tokenizer = pickle.load(handle)

In [71]:
# loading
with open('./vocabulary/en_tokenizer.pickle', 'rb') as handle:
    en_tokenizer = pickle.load(handle)

In [157]:
# split a text into sentences
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents

In [143]:
en_fr[0]

array(['I like tea'], dtype='<U10')

#### Translate

In [140]:
Text = 'I like tea'

In [141]:
en_fr = to_lines(Text)
en_fr = array(en_fr)

In [144]:
# Remove punctuation
en_fr[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in en_fr[:,0]]

In [145]:
en_fr

array([['I like tea']], dtype='<U10')

In [146]:
for i in range(len(en_fr)):
    en_fr[i,0] = en_fr[i,0].lower()

In [147]:
en_fr

array([['i like tea']], dtype='<U10')

In [148]:
# empty lists
en_l = []

# populate the lists with sentence lengths
for i in en_fr[:,0]:
    en_l.append(len(i.split()))

In [149]:
length_df = pd.DataFrame({'en':en_l})

In [150]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [151]:
# prepare validation data
toTranslate = encode_sequences(en_tokenizer, en_length, en_fr[:, 0])

In [152]:
toTranslate

array([[  1,  31, 379,   0,   0,   0,   0,   0]], dtype=int32)

In [153]:
# Predict
preds = model.predict_classes(toTranslate)

In [154]:
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [155]:
# convert predictions into text (French)
preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], fr_tokenizer)
        if j > 0:
            if (t == get_word(i[j-1], fr_tokenizer)) or (t == None):
                temp.append('')
            else:
                temp.append(t)
             
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t)            
        
    preds_text.append(' '.join(temp))

In [156]:
preds_text

['jaime le musique     ']

In [142]:
en_fr

array([['I like tea']], dtype='<U10')